In [1]:
import matplotlib
matplotlib.use("TkAgg")
import tqdm
import glob
from PIL import Image
import numpy as np
import itertools
import matplotlib.pyplot as plt
# noinspection PyUnresolvedReferences
import japanize_matplotlib
import time
# noinspection PyPackageRequirements
import umap
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_png(png_files):
    """
    指定されたフォルダ内のグレースケールpngファイルを読み込む
    :param png_files: glob文字列
    :return: 複数ファイルのpng配列
    """
    files = glob.glob(png_files)
    file_sample = Image.open(files[0]).convert('L')
    result_array = np.empty((len(files), file_sample.width, file_sample.height))
    count = 0
    pbar = tqdm.tqdm(files)
    pbar.set_description(png_files)
    for file in pbar:
        result_array[count] = np.array(Image.open(file).convert('L')).astype('float32') / 255
        count += 1
    return result_array

In [3]:
def load_png_all(paths):
    """
    複数フォルダのpngファイルを読み込む
    :param paths: (label文字列, ファイルglob文字列) pairの配列
    :return: (png配列, label配列)
    """
    result_labels = np.empty(0)
    result_datum = np.empty((0,701,701))
    for label, path in paths.items():
        result_data = load_png(path)
        result_datum = np.concatenate((result_datum, result_data))
        result_labels = np.concatenate((result_labels, np.full(result_data.shape[0], label)))
    return result_datum, result_labels

In [4]:
def show_sample(images):
    """
    複数のpngデータのプレビュー
    :param images: 複数のpng配列
    :return: なし
    """
    num = len(images)
    fig = plt.figure(figsize=(num * 4, 4))
    count = 1
    for title, image in images.items():
        axis = fig.add_subplot(1, num, count)
        plt.title(title)
        axis.imshow(image, cmap='gray', vmin=0, vmax=1)
        count += 1
    plt.show(block=False)

In [5]:
def show_result(umap_result, umap_labels):
    """
    UMAP結果を3D散布図で表示する
    :param umap_result: UMAP演算結果
    :param umap_labels: UMAP対象ラベル配列
    :return: なし
    """
    x = umap_result.embedding_[:, 0]
    y = umap_result.embedding_[:, 1]
    z = umap_result.embedding_[:, 2]
    color_cycle= itertools.cycle(["red","green","blue","orange","grey","brown","pink","yellow"])
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    for label in np.unique(umap_labels):
        ax.scatter(
            x[umap_labels == label],
            y[umap_labels == label],
            z[umap_labels == label],
            c=next(color_cycle), label=label
        )
    ax.legend(loc='upper left')
    plt.show(block=False)

In [6]:
labels = ['MP掌側', 'MP背側', '手関節背側']
png_datum, png_labels = load_png_all({label:f'data/2021_02_21 US 4クラス/{label}/*.png' for label in labels})
show_sample({label:png_datum[png_labels == label][0] for label in labels})

data/2021_02_21 US 4クラス/手関節背側/*.png: 100%|██████████| 300/300 [00:04<00:00, 65.76it/s]


In [7]:
t1 = time.time()
result3d = umap.UMAP(n_neighbors=5, n_components=3).fit(np.reshape(png_datum, (png_datum.shape[0], -1)))
print(f"経過時間:{time.time()-t1}")
show_result(result3d, png_labels)

経過時間:168.00667810440063
